In [169]:
import networkx

#Create Graph
def generateGraph(graphNumber):
    testGraph = networkx.MultiDiGraph()
    testGraph.add_node('Item' + str(graphNumber), value = '')
    testGraph.add_node('ItemName' + str(graphNumber), value = '')
    testGraph.add_node('ItemAffordance' + str(graphNumber), value = '')
    testGraph.add_node('ItemDescription' + str(graphNumber), value = '')
    testGraph.add_node('ItemRole' + str(graphNumber), value = '')

    testGraph.add_edge('Item' + str(graphNumber), 'ItemName' + str(graphNumber), value = 'ItemHasName')
    testGraph.add_edge('Item' + str(graphNumber), 'ItemAffordance' + str(graphNumber), value = 'ItemHasAffordance')
    testGraph.add_edge('Item' + str(graphNumber), 'ItemDescription' + str(graphNumber), value = 'ItemHasDescription')
    testGraph.add_edge('Item' + str(graphNumber), 'ItemRole' + str(graphNumber), value = 'ItemHasRole')

    return testGraph

In [170]:
class ItemGraph(object):
    #Constructor
    def __init__(self, graphNumber):
        self.graphNumber = graphNumber
        if(graphNumber != None):
            self.graph = generateGraph(self.graphNumber)
        else:
            self.graph = None
        
    #Generic append method based on whatever target is passed in    
    def __append(self, target, newValue):
        currentValue = self.graph.nodes(data=True)[target + str(self.graphNumber)]['value']
        if(currentValue == ''):
            updatedValue = newValue
        else:
            updatedValue = currentValue + '|' + newValue
        self.graph.nodes(data=True)[target + str(self.graphNumber)]['value'] = updatedValue
    
    #Generic replace method based on whatever target is passed in
    def __replace(self, target, newValue):
        self.graph.nodes(data=True)[target + str(self.graphNumber)]['value'] = newValue

    #Append/replace methods for each node value in Item Graph
    def appendItemValue(self, newValue):
        self.__append('Item', newValue)
        
    def replaceItemValue(self, newValue):
        self.__replace('Item', newValue)
                
    def appendItemName(self, newName):
        self.__append('ItemName', newName)
        
    def replaceItemName(self, newName):
        self.__replace('ItemName', newName)
        
    def appendItemAffordance(self, newAffordance):
        self.__append('ItemAffordance', newAffordance)
    
    def replaceItemAffordance(self, newAffordance):
        self.__replace('ItemAffordance', newAffordance)
    
    def appendItemDescription(self, newDescription):
        self.__append('ItemDescription', newDescription)
    
    def replaceItemDescription(self, newDescription):
        self.__replace('ItemDescription', newDescription)
    
    def appendItemRole(self, newRole):
        self.__append('ItemRole', newRole)
    
    def replaceItemRole(self, newRole):
        self.__replace('ItemRole', newRole)
        
    #Method to find a node containing a given value
    def FindItemWithValue(self, valueToFind):
        if(self.graph != None):
            #iterate through all graph nodes
            for node, values in self.graph.nodes.data():
                #If the current Node's value = the value passed in
                if(values['value'] == valueToFind):
                    return node
        return None
    
    #Methods to add different types of edges between nodes
    def addGroupMembershipEdges(self, groupNode, memberNode):
        self.graph.add_edge(memberNode, groupNode, value='IsMemberOf')
        self.graph.add_edge(groupNode, memberNode, value='HasMember')
        
    def addNodeEquivalencyEdges(self, firstNode, secondNode):
        self.graph.add_edge(firstNode, secondNode, value='IsEquivalentTo')
        self.graph.add_edge(secondNode, firstNode, value='IsEquivalentTo')
        
    def addCompositionEdges(self, composedNode, partOfNode):
        self.graph.add_edge(composedNode, partOfNode, value='HasA')
        self.graph.add_edge(partOfNode, composedNode, value='IsPartOf')
    
    #Methods to replace values of specific nodes
    def ReplaceItemAffordanceAtSpecificNode(self, nodeToAddAffordance, newAffordance):
        node = self.FindItemWithValue(nodeToAddAffordance)
        if(node != None):
            edgesFromNode = self.graph.edges(node, data=True)
            for startNode, endNode, edgeValues in edgesFromNode:
                #If an edge has the value ItemHasName, then we want to modify the end node
                if(edgeValues['value'] == 'ItemHasAffordance'):
                    #Update graph with name
                    self.graph.nodes(data=True)[endNode]['value'] = newAffordance
                    return True
        else:
            print("No node with direct object reference as value found")
            return False

    def ReplaceItemNameAtSpecificNode(self, nodeToAddName, newName):
        #Find Node
        node = self.FindItemWithValue(nodeToAddName)
        if(node != None):
            #Get list of edges from the node
            edgesFromNode = self.graph.edges(node, data=True)
            for startNode, endNode, edgeValues in edgesFromNode:
                #If an edge has the value ItemHasName, then we want to modify the end node
                if(edgeValues['value'] == 'ItemHasName'):
                    #Update graph with name
                    self.graph.nodes(data=True)[endNode]['value'] = newName
                    return True
        else:
            print("No node with direct object reference as value found")
            return False

In [171]:
class predicateSwitcher(object):
    
    def __init__(self):
        self.graphNumber = 0
        self.DRSGraph = ItemGraph(None)
        
    #Method to call the appropriate function based on the argument passed in
    def callFunction(self, predicateType, predicateContents):
        #Get the name of the method
        methodName = 'predicate_' + str(predicateType)
        #Get the method itself
        method = getattr(self, methodName, lambda: "Unknown predicate")
        #Call the method and return its output
        method(predicateContents)
        return self.DRSGraph
        
    #For object() predicates
    #SHOULD CHECK IF OBJECT WITH GIVEN NAME ALREADY EXISTS!!!  IF SO, FIGURE OUT WHAT ARE THE CONDITIONS FOR THAT TO OCCUR
    def predicate_object(self, predicateContents):
        print(predicateContents)
        #Break up elements of object line into variables
        predicateComponents = predicateContents.split(',')
        objReferenceVariable = predicateComponents[0]
        objName = predicateComponents[1]
        objClass = predicateComponents[2]
        objUnit = predicateComponents[3]
        objOperator = predicateComponents[4]
        objCount = predicateComponents[5].split(')')[0]
        if(self.DRSGraph.FindItemWithValue(objReferenceVariable) == None):
            #Apply appropriate variables to ItemGraph
            objectGraph = ItemGraph(self.graphNumber)
            objectGraph.appendItemValue(objReferenceVariable)
            objectGraph.appendItemRole(objName)
            #Increase the graph number for auto-generation of names
            self.graphNumber = self.graphNumber + 1
            #If a main graph already exists, then add the new graph in to it
            if(self.DRSGraph.graph != None):
                #DRSGraph.graph = networkx.algorithms.operators.binary.union(DRSGraph.graph, objectGraph.graph)
                self.DRSGraph.graph = networkx.algorithms.operators.binary.compose(self.DRSGraph.graph, objectGraph.graph)
            #if no main graph exists, this is the main graph
            else:
                self.DRSGraph.graph = objectGraph.graph
            return True
        else:
            return False
    
    #For predicate() predicates
    #HOW TO HANDLE SENTENCE SUB-ORDINATION?
    def predicate_predicate(self, predicateContents):
        #Intransitive verbs: (predName, verb, subjectRef) 
        #- The SubjectRef Verbed (the man laughed, the target appears)
        #Transitive verbs: (predName, verb, subjectRef, dirObjRef)
        #- The Subjectref Verbed the dirObjRef (the task A has a group of objects H, the subject L remembers the letter I)
        #Ditransitive verbs: (predName, verb, subjRef, dirObjRef, indirObjRef)
        #- The SubjectRef verbed the DirObjRef to the indirObjRef (The professor (S) gave the paper (D) to the student (I))
        #Break up the predicate
        predicateComponents = predicateContents.split(',')
        numberOfComponents = len(predicateComponents)
        #Always have first three components, so only special cases are transitive/ditransitive
        predReferenceVariable = predicateComponents[0]
        predVerb = predicateComponents[1]
        predSubjRef = predicateComponents[2]
        #Different cases (differing number of components)
        if(numberOfComponents == 3):
            #intransitive
            predSubjRef = predSubjRef.split(')')[0]
        elif(numberOfComponents == 4):
            #Transitive
            predDirObjRef = predicateComponents[3].split(')')[0]
        elif(numberOfComponents == 5):
            #Ditransitive
            predIndirObjRef = predicateComponents[4].split(')')[0]
        else:
            #invalid
            raise ValueError('Too many components ?')
        #Hardcode be case for specific scenarios
        if(predVerb == 'be'):
            #Check if naming or setting an equivalency
            if('named' in predSubjRef):
                #If so call naming method
                self.DRSGraph = nameItem(predSubjRef, predDirObjRef, self.DRSGraph)
            #If not named(XYZ) but still has 4 components
            elif(numberOfComponents == 4):
                #Get nodes for both subject and direct object
                subjRefNode = self.DRSGraph.FindItemWithValue(predSubjRef)
                dirObjRefNode = self.DRSGraph.FindItemWithValue(predDirObjRef)
                #If both are nodes in the graph, then the "Be" is setting an equivalency
                if(subjRefNode != None and dirObjRefNode != None):
                    self.DRSGraph.addNodeEquivalencyEdges(subjRefNode, dirObjRefNode)
            #HANDLE ANY OTHER CASES????
        #Hardcode "have" case for composition
        elif(predVerb == 'have'):
            if(numberOfComponents == 4):
                #Get nodes for both subject and direct object
                subjRefNode = self.DRSGraph.FindItemWithValue(predSubjRef)
                dirObjRefNode = self.DRSGraph.FindItemWithValue(predDirObjRef)
                #If both are nodes in the graph, then the "have" is setting a composition
                if(subjRefNode != None and dirObjRefNode != None):
                    self.DRSGraph.addCompositionEdges(subjRefNode, dirObjRefNode)
        else:
            #If just one subject "The target appears"
            if(numberOfComponents == 3):
                self.DRSGraph.ReplaceItemAffordanceAtSpecificNode(predSubjRef, predVerb)
            #HOW TO HANDLE INTERACTIONS (the subject remembers the letter)
            #TODO TODO TODO TODO
            elif(numberOfComponents == 4):
                pass
            #TODO TODO TODO TODO
            elif(numberOfComponents == 5):
                pass
                
    #For has_part() predicates
    def predicate_has_part(self, predicateContents):
        #Get predicate items
        predicateComponents = predicateContents.split(',')
        predGroupRef = predicateComponents[0]
        predGroupMember = predicateComponents[1].split(')')[0]
        #Hardcode the new object as being a group
        predGroupDescription = 'GROUP'
        #if Group reference doesn't exist
        groupNode = self.DRSGraph.FindItemWithValue(predGroupRef)
        memberNode = self.DRSGraph.FindItemWithValue(predGroupMember)
        if(groupNode == None):
             #Then create that item
             #Apply appropriate variables to ItemGraph
            groupGraph = ItemGraph(self.graphNumber)
            groupGraph.appendItemValue(predGroupRef)
            groupGraph.appendItemRole(predGroupDescription)
            #Get the node for the group
            groupNode = groupGraph.FindItemWithValue(predGroupRef)
            #Increase the graph number for auto-name generation
            self.graphNumber = self.graphNumber + 1
            #Compose the new graph with the existing graph 
            #(no scenario of no existing graph because can't start with has_part())
            self.DRSGraph.graph = networkx.algorithms.operators.binary.compose(self.DRSGraph.graph, groupGraph.graph)
        #Add membership edges
        self.DRSGraph.addGroupMembershipEdges(groupNode, memberNode)
    
    #HANDLE MODIFIERS
    #TODO TODO TODO TODO
    def predicate_modifier_pp(self, predicateContents):
        pass
    
    #HANDLE PROPERTIES
    #TODO TODO TODO TODO
    def predicate_property(self, predicateContents):
        pass

In [172]:
#CAN PROBABLY MAKE THE NAME CHANGE A CLASS FUNCTION ????  MAKE APPEND/REPLACE BE ABLE TO TAKE IN SPECIFIC TARGETS?
def nameItem(predSubjRef, predDirObjRef, DRSGraph):
    #Get item name out of "named(XYZ)"
    itemName = predSubjRef[predSubjRef.find("(")+1:predSubjRef.find(")")]
    #Replace the name
    DRSGraph.ReplaceItemNameAtSpecificNode(predDirObjRef, itemName)
    #Return graph
    return DRSGraph

In [173]:
#Check if the current line is a predicate or not
def isLinePredicate(inputDRS):
    #Check if starts with bracket (header) or is arrow (conditional)
    if(inputDRS[0] == '[' or inputDRS == '=>'):
        return False
    else:
        return True

In [174]:
#Check if current line is a conditional or a header line
def getConditionalLines(DRSLines):
    #Declarations
    headerLines = []
    conditionalLines = []
    #Iterate through DRS commands
    for index, line in enumerate(DRSLines):
        #If line starts with bracket, it's a header
        if(line[0] == '['):
            headerLines.append(index)            
        #if line is arrow, it's a conditional
        elif(line == '=>'):
            conditionalLines.append(index)
    return headerLines, conditionalLines

In [175]:
#Check if current line is in the "if" part of a conditional
#SOMEHOW VERY WRONG, NEEDS FIXED
def isLineConditionalIf(lineIndex, headerLines, conditionalLines):
    #Declarations
    mostRecentHeader = 0
    mostRecentConditional = 0
    #Iterate through all indexes of header lines
    for header in headerLines:
        #If the current header is less than or equal to the line index we are interested in
        #And the header is greater than the max header so far
        if(header <= lineIndex and header > mostRecentHeader):
            #Update the max header
            mostRecentHeader = header
            
    #Iterate through all indexes of conditional lines
    for conditional in conditionalLines:
        #If the current conditional is less than or equal to the line index we are interested in
        #And the current conditional is greater than the max conditional so far
        if(conditional <= lineIndex and conditional > mostRecentConditional):
            #Update the max conditional
            mostRecentConditional = conditional
            
    #THE CURRENT LINE IS PART OF THE "IF" PART OF A CONDITIONAL IN THE CASE THAT:
    #The max header before the line index is less than the current line index (REDUNDANT???) AND
    #The max conditional before the current line index is less than the max header before the line index
    if(mostRecentHeader < lineIndex and mostRecentConditional < mostRecentHeader):
        return True
    else:
        return False

In [176]:
#Check if current line is in the "then" part of a conditional
#SOMEHOW VERY WRONG, NEEDS FIXED
def isLineConditionalThen(lineIndex, headerLines, conditionalLines):
    #Declarations
    numberOfHeadersSinceLastConditional = 0
    mostRecentConditional = 0
    #Iterate through all indexes of conditionals
    for conditional in conditionalLines:
        #If the current conditional index is less than or equal to the line index AND 
        #The current conditional index is greater than the max conditional index so far
        if(conditional <= lineIndex and conditional > mostRecentConditional):
            #Update the max conditional
            mostRecentConditional = conditional
            
    #Iterate through all indexes of headers        
    for header in headerLines:
        #If the current header is greater than the max conditional
        #AND the current header is less than the line index
        if(header > mostRecentConditional and header < lineIndex):
            #Update how many headers have passed since the most recent conditional
            numberOfHeadersSinceLastConditional = numberOfHeadersSinceLastConditional + 1
    #If only one header has occured since the last conditional, then we are in a "then"
    if(numberOfHeadersSinceLastConditional == 1):
        return True
    else:
        return False

In [181]:
#how to handle interactions  (target appears in box, etc.)
#how to handle as of yet unhandled predicates
def DRSToItem():
    #Declare relevant variables
    DRSGraph = None
    DRSLines = []
    #Read in DRS instructions from file
    DRSFile = open("DRS_read_in.txt", "r")
    for line in DRSFile:
        #Get DRS command and remove any leading and ending whitespace
        DRSLines.append(line.strip())
    #Get numbers of which lines are headers ([A, B, C, ...] and conditionals (=>) )
    headerLines, conditionalLines = getConditionalLines(DRSLines)
    #Set up the predicate switcher
    switcher = predicateSwitcher()
    #while(nextStep != 'exit'):
    
    #Iterate through the DRS instructions
    for index, currentInstruction in enumerate(DRSLines):
        print(currentInstruction)
        #Find out if the current line is part of a conditional IF or not
        conditionalIf = isLineConditionalIf(index, headerLines, conditionalLines)
        if(conditionalIf == True):
            print("IF LINE")
            
        #Find out if the current line is part of a conditional THEN or not
        conditionalThen = isLineConditionalThen(index, headerLines, conditionalLines)
        if(conditionalThen == True):
            print("THEN LINE")
        #take next instruction or exit
        nextStep = input("Enter anything to read in next line\n")
        
        #As long as no "exit" given
        if(nextStep != 'exit'):
            #print(currentInstruction)
            #print(conditional)
            #Find out if the current line is a predicate or not
            predicate = isLinePredicate(currentInstruction)
            #If it is a predicate
            if(predicate == True):
                #Get the predicate type and contents
                predicateSplit = currentInstruction.split('(', 1)
                predicateType = predicateSplit[0]
                predicateContents = predicateSplit[1]
                print(predicateType)
                #Call appropriate handling function based on predicate type
                DRSGraph = switcher.callFunction(predicateType, predicateContents)
                #Draw the current form of the ontology
                networkx.draw(DRSGraph.graph, labels=networkx.get_node_attributes(DRSGraph.graph, 'value'))
                plt.show()
            else:
                #Handle headers
                pass
        #Break out of loop with exit
        else:
            break
    #At end of program, if an ontology was built at all, print it out and export it in GraphML
    if(DRSGraph != None):
        networkx.draw(DRSGraph.graph, labels=networkx.get_node_attributes(DRSGraph.graph, 'value'))
        plt.show()
        networkx.write_graphml_lxml(DRSGraph.graph, "DRSGraph.graphml")

In [ ]:
DRSToItem()